# Fichiers proposés par Sandrine Edouard
# TEAPD-2599 Convertisseur pour matrices Scribe: Analyse

But du notebook: Explorer les fichiers grib envoyés par Sandrine Édouard en 2019. Ces fichiers grib contiennent la totalité des champs météorologiques à aller chercher.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import cftime
import datetime
import xarray as xr
import wget
import os
import glob
import matplotlib.pyplot as plt
import dask
import cartopy.crs as ccrs
%matplotlib inline
import multiprocessing.popen_spawn_posix
from dask.distributed import Client

In [10]:

client = Client(n_workers=1, threads_per_worker=1, memory_limit='2GB')
client

/Users/caramelo/Documents/python-virtual-environments/rdpsgrib/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52479 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:52480 Dashboard: http://127.0.0.1:52479/status,Cluster Workers: 1 Cores: 1 Memory: 2.00 GB


Question: Combien de fichier Grib en simultané on peut prendre? Est-ce que les fichiers envoyés par Sandrine sont déjà coupés?

In [11]:
os.chdir('/Users/caramelo/Documents/0000_Unu_Engineering/Grib Sandrine/2019010800-06-12-18_000-048_hrdps/')
grib_list_total=sorted(glob.glob('*00_*.grib2'))
grib_list=grib_list_total[0:7]
#grib_list=grib_list_total
grib_list

['2019010800_001.grib2',
 '2019010800_002.grib2',
 '2019010800_003.grib2',
 '2019010800_004.grib2',
 '2019010800_005.grib2',
 '2019010800_006.grib2',
 '2019010800_007.grib2']

Penser à lancer une requête dans grib_api voir ce qu'il y a exactement dans chaque fichier grib. C'est à dire combien de messages sont contenus dans chaque.

Les fichiers de base ont deux niveaux verticaux donc il faut utiliser ceci comme backend_kwargs: 

    filter_by_keys={'typeOfLevel': 'surface'}
    filter_by_keys={'typeOfLevel': 'heightAboveGround'

In [12]:
ds_surface=xr.open_mfdataset(grib_list,concat_dim='valid_time',engine='cfgrib',combine='nested',parallel=True,backend_kwargs={'filter_by_keys':{'typeOfLevel':'surface'}})

In [13]:
ds_surface

<xarray.Dataset>
Dimensions:     (valid_time: 7, x: 2576, y: 1456)
Coordinates:
    time        datetime64[ns] 2019-01-08
    step        (valid_time) timedelta64[ns] 01:00:00 02:00:00 ... 07:00:00
    surface     int64 0
    latitude    (y, x) float64 dask.array<chunksize=(1456, 2576), meta=np.ndarray>
    longitude   (y, x) float64 dask.array<chunksize=(1456, 2576), meta=np.ndarray>
  * valid_time  (valid_time) datetime64[ns] 2019-01-08T01:00:00 ... 2019-01-0...
Dimensions without coordinates: x, y
Data variables:
    paramId_0   (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
    gust        (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
    pop         (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
    tstm        (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-03-05T15:12:40 GRIB to CDM+CF via cfgrib-0....

In [14]:
ds_heightAboveGround=xr.open_mfdataset(grib_list,concat_dim='valid_time',engine='cfgrib',combine='nested',parallel=True,backend_kwargs={'filter_by_keys':{'typeOfLevel':'heightAboveGround'}})

In [15]:
ds_heightAboveGround

<xarray.Dataset>
Dimensions:            (valid_time: 7, x: 2576, y: 1456)
Coordinates:
    time               datetime64[ns] 2019-01-08
    step               (valid_time) timedelta64[ns] 01:00:00 ... 07:00:00
    heightAboveGround  int64 2
    latitude           (y, x) float64 dask.array<chunksize=(1456, 2576), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(1456, 2576), meta=np.ndarray>
  * valid_time         (valid_time) datetime64[ns] 2019-01-08T01:00:00 ... 20...
Dimensions without coordinates: x, y
Data variables:
    paramId_0          (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
    d2m                (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
    t2m                (valid_time, y, x) float32 dask.array<chunksize=(1, 1456, 2576), meta=np.ndarray>
Attributes:
    GRIB_edition:            2
    GRIB_centre:             cwao
    GRIB_centreDescription:  Canadian Meteorological Service - Montreal 
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Canadian Meteorological Service - Montreal 
    history:                 2021-03-05T15:35:43 GRIB to CDM+CF via cfgrib-0....

In [ ]:
client.close()